# Raster Processing Workflow
**Cara Piske, Graduate Program of Hydrologic Sciences, 2022; Advisor: Dr. Adrian Harpold**<br>
<p>This code processes rasterized LAS files. <br>
Lidar data were provided by the Airborne Snow Observatory (ASO), the National Center for Airborne Laser Mapping (NCALM), and Watershed Sciences Inc. (WSI). <br>

The goal of this project is to process snow depth to the one-meter spatial scale while maintaining conservative under-canopy estimates. Therefore, little interpolation occurs under-canopy. We follow these protocols in order to obtain a 1-m rasterized product (as opposed to the 3-m rasterized product provided by ASO on the NSIDC data portal). NCALM and WSI flights were obtained through OpenTopography.

In [1]:
# import necessary packages 
from osgeo import gdal, ogr, osr
import csv
import os
import subprocess
import sys
import pdal
import matplotlib.pyplot as plt
import numpy as np
import json
import glob
import time
#import pycrown
# competing paths on our PC 
gdal_merge = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_merge.py')
gdal_calc = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_calc.py')
gdal_warp = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_warp.py')
gdal_polygonize = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_polygonize.py')
gdal_proximity = os.path.join('C:\\','Users','cpiske','.conda','envs','lidar','Lib','site-packages','osgeo_utils','gdal_proximity.py')

In [2]:
#make sure we're in the right working directory
os.chdir('/')
print(os.getcwd())

G:\


## Classify Vegetation
We use vegetation classifications based on specific strata. See: https://docs.google.com/document/d/1-HndVgydwBWQGCzLMCvowkY_Idll0IDog-vd3OrTiio/edit?usp=sharing

In [20]:
# assign var names
# Number of Returns in each HAG strata
strata_neg0pt15_0pt15 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_neg0pt15_0pt15.tif'
strata_0pt15_1pt5 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_0pt15_1pt5.tif'
strata_1pt5_3 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_1pt5_3.tif'
strata_3 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_3.tif'
# Filter based off criteria for vegetation classes, files are binary
counts_filtered_path = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/counts_filtered/'
strata_neg0pt15_0pt15_GT0 = counts_filtered_path + 'strata_neg0pt15_0pt15_GT0.tif'
strata_neg0pt15_0pt15_GTET0 = counts_filtered_path + 'strata_neg0pt15_0pt15_GTET0.tif'
strata_0pt15_1pt5_GT0 = counts_filtered_path + 'strata_0pt15_1pt5_GT0.tif'
strata_0pt15_1pt5_ET0 = counts_filtered_path + 'strata_0pt15_1pt5_ET0.tif'
strata_1pt5_3_ET0 = counts_filtered_path + 'strata_1pt5_3_ET0.tif'
strata_3_GT0 = counts_filtered_path + 'strata_3_GT0.tif'
strata_3_ET0 = counts_filtered_path + 'strata_3_ET0.tif'

# Vegetation Classes, files are binary
ncalm_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_open.tif'
ncalm_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall.tif'
ncalm_short = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_short.tif'
ncalm_understory = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_understory.tif'

ncalm_tallORunderstory = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tallORunderstory.tif'
ncalm_openORshort = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_openORshort.tif'

ncalm_short_HAG = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_short_HAG.tif'
ncalm_understory_HAG = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_understory_HAG.tif'

CHM_LT1pt5 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM_LT1pt5.tif'
CHM = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM.tif'

# veg_OpenTall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/veg_strata/NCALM_SCB_20201120_tallORopen.tif'
# veg_total = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/veg_strata/NCALM_SCB_20201120_shortORtallORopen.tif'

veg_classified = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_vegClassified.tif'

We'll start off by looking at the number of returns in each height strata and filtering based off of our criteria. <br>

In [4]:
# nor_filtered_cmd_a = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15, '--outfile', strata_neg0pt15_0pt15_GT0, '--calc="A>0"','--overwrite']
# nor_filtered_cmd_b = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15, '--outfile', strata_neg0pt15_0pt15_GTET0, '--calc="A>=0"','--overwrite']
# nor_filtered_cmd_c = ['python', gdal_calc, '-A', strata_0pt15_1pt5, '--outfile', strata_0pt15_1pt5_GT0, '--calc="A>0"','--overwrite']
# nor_filtered_cmd_d = ['python', gdal_calc, '-A', strata_0pt15_1pt5, '--outfile', strata_0pt15_1pt5_ET0, '--calc="A==0"','--overwrite']
# nor_filtered_cmd_e = ['python', gdal_calc, '-A', strata_1pt5_3, '--outfile', strata_1pt5_3_ET0, '--calc="A==0"','--overwrite']
# nor_filtered_cmd_f = ['python', gdal_calc, '-A', strata_3, '--outfile', strata_3_ET0, '--calc="A==0"','--overwrite']
# nor_filtered_cmd_g = ['python', gdal_calc, '-A', strata_3, '--outfile', strata_3_GT0, '--calc="A>0"','--overwrite']

# subprocess.run(nor_filtered_cmd_a)
# subprocess.run(nor_filtered_cmd_b)
# subprocess.run(nor_filtered_cmd_c)
# subprocess.run(nor_filtered_cmd_d)
# subprocess.run(nor_filtered_cmd_e)
# subprocess.run(nor_filtered_cmd_f)
# subprocess.run(nor_filtered_cmd_g)

### Open, Tall, Short Veg, Understory
After filtering based of strata criteria, we can classify the snow-off flight into vegetation classifications. <br> 


In [7]:
# it's necessary for the NaN to be 0 here because we eventually multiply these rasters by snow rasters and 
# we don't want artificial 0's
open_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GT0, '-B', strata_0pt15_1pt5_ET0,'-C',strata_1pt5_3_ET0,'-D',strata_3_ET0,'--NoDataValue','0',
 '--outfile', ncalm_open, '--calc="A*B*C*D"','--overwrite']
subprocess.run(open_cmd)

tall_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GTET0, '-B', strata_0pt15_1pt5_ET0,'-C',strata_1pt5_3_ET0,'-D',strata_3_GT0,'--NoDataValue','0',
 '--outfile', ncalm_tall, '--calc="A*B*C*D"','--overwrite']
subprocess.run(tall_cmd)

short_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GTET0, '-B', strata_0pt15_1pt5_GT0,'-C',strata_1pt5_3_ET0,'-D',strata_3_ET0,'--NoDataValue','0',
 '--outfile', ncalm_short, '--calc="A*B*C*D"','--overwrite']
subprocess.run(short_cmd)

understory_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GTET0, '-B', strata_0pt15_1pt5_GT0,'-C',strata_1pt5_3_ET0,'-D',strata_3_GT0,'--NoDataValue','0',
 '--outfile', ncalm_understory, '--calc="A*B*C*D"','--overwrite']
subprocess.run(understory_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/counts_filtered/strata_neg0pt15_0pt15_GTET0.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/counts_filtered/strata_0pt15_1pt5_GT0.tif', '-C', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/counts_filtered/strata_1pt5_3_ET0.tif', '-D', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/counts_filtered/strata_3_GT0.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_understory.tif', '--calc="A*B*C*D"', '--overwrite'], returncode=0)

In [ ]:
# Not necessary
# # now we want nan to be -9999
# open_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GT0, '-B', strata_0pt15_1pt5_ET0,'-C',strata_1pt5_3_ET0,'-D',strata_3_ET0,'--NoDataValue','-9999',
#  '--outfile', ncalm_open_nan9999, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(open_cmd)

# tall_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GTET0, '-B', strata_0pt15_1pt5_ET0,'-C',strata_1pt5_3_ET0,'-D',strata_3_GT0,'--NoDataValue','-9999',
#  '--outfile', ncalm_tall_nan9999, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(tall_cmd)

# short_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GTET0, '-B', strata_0pt15_1pt5_GT0,'-C',strata_1pt5_3_ET0,'-D',strata_3_ET0,'--NoDataValue','-9999',
#  '--outfile', ncalm_short_nan9999, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(short_cmd)

# understory_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15_GTET0, '-B', strata_0pt15_1pt5_GT0,'-C',strata_1pt5_3_ET0,'-D',strata_3_GT0,'--NoDataValue','-9999',
#  '--outfile', ncalm_understory_nan9999, '--calc="A*B*C*D"','--overwrite']
# subprocess.run(understory_cmd)

Now let's extract the HAG values at potential short/understory locations

In [12]:
short_cmd = ['python', gdal_calc, '-A', ncalm_short, '-B', CHM_LT1pt5,
 '--outfile', ncalm_short_HAG, '--calc="B*A"','--overwrite','--NoDataValue','-9999']
subprocess.run(short_cmd)

understory_cmd = ['python', gdal_calc, '-A', ncalm_understory, '-B', CHM_LT1pt5,'--NoDataValue','-9999',
 '--outfile', ncalm_understory_HAG, '--calc="A*B"','--overwrite']
subprocess.run(understory_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_short.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM_LT1pt5.tif', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/NCALM_SCB_20201120_short_HAG.tif', '--calc="B*A"', '--overwrite', '--NoDataValue', '-9999'], returncode=0)

In [23]:
# tall or understory
tall_cmd = ['python', gdal_calc, '-A', ncalm_tall, '-B', ncalm_understory,
 '--outfile', ncalm_tallORunderstory, '--calc="B+A"','--overwrite','--hideNoData','--NoDataValue','0']
subprocess.run(tall_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_understory.tif', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tallORunderstory.tif', '--calc="B+A"', '--overwrite', '--hideNoData', '--NoDataValue', '0'], returncode=0)

In [24]:
# short or open
short_cmd = ['python', gdal_calc, '-A', ncalm_open, '-B', ncalm_short,
 '--outfile', ncalm_openORshort, '--calc="B+A"','--overwrite','--NoDataValue','0']
subprocess.run(short_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_open.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_short.tif', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_openORshort.tif', '--calc="B+A"', '--overwrite', '--NoDataValue', '0'], returncode=0)

In [34]:
# classify all veg

veg_cmd = ['python', gdal_calc, '-A', ncalm_open, '-B', ncalm_short, '-C', ncalm_understory, '-D', ncalm_tall,
 '--outfile', veg_classified, '--calc="A+(2*B)+(3*C)+(4*D)"','--overwrite', '--hideNoData','--NoDataValue','-9999']
subprocess.run(veg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_open.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_short.tif', '-C', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_understory.tif', '-D', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall.tif', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_vegClassified.tif', '--calc="A+(2*B)+(3*C)+(4*D)"', '--overwrite', '--hideNoData', '--NoDataValue', '-9999'], returncode=0)

#### Filter
water bodies

In [35]:
input_shp = 'SCB/supporting_files/masks/NHDWaterbody_2018_EPSG26910.shp'
input_list = [ncalm_open, ncalm_tall, ncalm_short, ncalm_understory, ncalm_tallORunderstory, ncalm_openORshort]
for input_tif in input_list:
    burn_command = ['gdal_rasterize', '-b', '1', '-burn', '0', input_shp, input_tif]
    subprocess.call(burn_command)

input_listb = [ncalm_short_HAG, ncalm_understory_HAG, veg_classified]
for input_tif in input_listb:
    burn_command = ['gdal_rasterize', '-b', '1', '-burn', '0', input_shp, input_tif]
    subprocess.call(burn_command)

### Characterize Veg

In [36]:
# Filter out all water bodies
input_shp = 'SCB/supporting_files/masks/NHDWaterbody_2018_EPSG26910.shp'

In [37]:
#ncalm_tallORunderstory = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tallORunderstory.tif'
dist_to_nearest_canopy = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_DNC.tif'
dist_to_nearest_canopy_30 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_DNC_30m.tif'
dist_to_nearest_canopy_10 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_2020_DNC_10m.tif'

clump_size = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_clumpSize.tif'
clump_size_30 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_clumpSize_30m.tif'
clump_size_10 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_clumpSize_10m.tif'

#### DNC

In [38]:
# extract vegetation cover (average height and density) across the watershed
proximity_cmd = ['python',gdal_proximity, ncalm_tallORunderstory, dist_to_nearest_canopy,'-values','1','-nodata','-9999']
subprocess.run(proximity_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_proximity.py', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tallORunderstory.tif', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_DNC.tif', '-values', '1', '-nodata', '-9999'], returncode=0)

In [39]:
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, dist_to_nearest_canopy]
subprocess.call(burn_command)

0

In [40]:
# extract vegetation cover (average height and density) across the watershed
# 'python',gdal_proximity
proximity_cmd = ['python',gdal_proximity, strata_3, clump_size,'-values','0','-nodata','-9999']
subprocess.run(proximity_cmd)
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, clump_size]
subprocess.call(burn_command)

0

In [41]:
warp_cmd = ["gdalwarp","-overwrite", clump_size, clump_size_30, "-tr", str(30),str(30),'-srcnodata','-9999','-r','average','-dstnodata','-9999']
subprocess.call(warp_cmd)

0

In [42]:
warp_cmd = ["gdalwarp","-overwrite", dist_to_nearest_canopy, dist_to_nearest_canopy_30, "-tr", str(30),str(30),'-srcnodata','-9999','-r','average','-dstnodata','-9999']
subprocess.call(warp_cmd)

0

#### fVeg and canopy density

In [47]:
ncalm_fVEG_30m = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/fVeg/NCALM_SCB_20201120_fVEG_30m.tif'

ncalm_CHM = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM.tif'
ncalm_CHM_gt3 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM_gt3_binary.tif'
ncalm_CHM_gt3_sum_30m = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM_gt3_sum_30m.tif'

ncalm_canopy_density_30m = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/fVeg/NCALM_SCB_20201120_canopy_density_30m.tif'
ncalm_canopy_density = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/fVeg/NCALM_SCB_20201120_canopy_density.tif'

strata_3_30m = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_3_30m.tif'

counts_total = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts.tif'
counts_total_GT0 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT0.tif'
counts_total_GT0_30 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT0_30m.tif'
counts_total_30 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_30m.tif'

In [48]:
# We need a raster containing a logical for count values gt 0 because there are locations where the count=0
# and we don't want to divide by 0
veg_cmd = ['python',gdal_calc, '-A', counts_total,'--NoDataValue','0',
 '--outfile', counts_total_GT0, '--calc="A*(A>0)"','--overwrite']
subprocess.run(veg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT0.tif', '--calc="A*(A>0)"', '--overwrite'], returncode=0)

In [49]:
veg_cmd = ['python',gdal_calc, '-A', counts_total_GT0, '-B', strata_3,'--NoDataValue','-9999',
 '--outfile', ncalm_canopy_density, '--calc="B/A"','--overwrite']
subprocess.run(veg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT0.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_3.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/fVeg/NCALM_SCB_20201120_canopy_density.tif', '--calc="B/A"', '--overwrite'], returncode=0)

In [50]:
# # use ts to specify the x and y extents of bounding box (default 1000)
warp_cmd = ["gdalwarp","-overwrite", counts_total, counts_total_30, "-tr", str(30),str(30),'-srcnodata','-9999','-r','sum','-dstnodata','-9999']
subprocess.call(warp_cmd)
# calculate canopy density
veg_cmd = ['python',gdal_calc, '-A', counts_total_30,'--NoDataValue','0',
 '--outfile', counts_total_GT0_30, '--calc="A*(A>0)"','--overwrite']
subprocess.run(veg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_30m.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT0_30m.tif', '--calc="A*(A>0)"', '--overwrite'], returncode=0)

In [51]:
warp_cmd = ["gdalwarp","-overwrite", strata_3, strata_3_30m, "-tr", str(30),str(30),'-srcnodata','-9999','-r','sum','-dstnodata','-9999']
subprocess.call(warp_cmd)

0

In [52]:
calc_cmd = ['python',gdal_calc, '-A', counts_total_GT0_30, '-B', strata_3_30m,'--NoDataValue','-9999',
 '--outfile', ncalm_canopy_density_30m, '--calc="B/A"','--overwrite']
subprocess.run(calc_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT0_30m.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_height_strata/vegStrata_3_30m.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/fVeg/NCALM_SCB_20201120_canopy_density_30m.tif', '--calc="B/A"', '--overwrite'], returncode=0)

**fVEG**

In [53]:
fVeg_cmd = ['python',gdal_calc, '-A', ncalm_CHM, '--NoDataValue','-9999',
 '--outfile', ncalm_CHM_gt3, '--calc="A>=3"','--overwrite']
subprocess.run(fVeg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM_gt3_binary.tif', '--calc="A>=3"', '--overwrite'], returncode=0)

In [54]:
# # use ts to specify the x and y extents of bounding box (default 1000)
# warp_cmd = ["gdalwarp","-overwrite", clump_size, clump_size_30, "-tr", str(30),str(30),'-srcnodata','-9999','-r','average','-dstnodata','-9999']
warp_cmd = ["gdalwarp","-overwrite", ncalm_CHM_gt3, ncalm_CHM_gt3_sum_30m, "-tr", str(30),str(30),'-srcnodata','-9999','-r','sum','-dstnodata','-9999']

subprocess.call(warp_cmd)

0

In [55]:
# # use ts to specify the x and y extents of bounding box (default 1000)
# warp_cmd = ["gdalwarp","-overwrite", dist_to_nearest_canopy, dist_to_nearest_canopy_30m, "-tr", str(30),str(30),'-srcnodata','-9999','-r','average','-dstnodata','-9999']
fVeg_cmd = ['python',gdal_calc, '-A', ncalm_CHM_gt3_sum_30m, '--NoDataValue','-9999',
 '--outfile', ncalm_fVEG_30m, '--calc="A/(30*30)"','--overwrite']
subprocess.run(fVeg_cmd)


CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM_gt3_sum_30m.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/fVeg/NCALM_SCB_20201120_fVEG_30m.tif', '--calc="A/(30*30)"', '--overwrite'], returncode=0)

#### TAO

See: produce_canopy_metrics.Rmd for TAO delineation <br>
Right now, this workflow uses QGIS to convert the TAO delineation from a csv-shp-tif, this could be done here as well (see: https://geopandas.org/en/stable/docs/user_guide/io.html) <br>

There are multiple metrics we derive from this product, one is average height/30m pixel, the other is canopy density/30m pixel

In [64]:
TAO_ras = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_TAO.tif' 
counts_ras = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_TAO_counts_30m.tif' 
avg_height_ras = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_avg_TOA_height_30m.tif'
max_height_ras = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_max_TOA_height_30m.tif'

TAO_ras_binary = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_TAO_binary.tif'
TAO_density = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_TAO_density_30m.tif'

LAI = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_LAI_index_30m.tif'


In [57]:
fVeg_cmd = ['python',gdal_calc, '-A', TAO_ras, '--NoDataValue','-9999',
 '--outfile', TAO_ras_binary, '--calc="1*A>0"','--overwrite']
subprocess.run(fVeg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_TAO.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_TAO_binary.tif', '--calc="1*A>0"', '--overwrite'], returncode=0)

In [58]:
warp_cmd = ["gdalwarp","-overwrite", TAO_ras_binary, counts_ras, "-tr", str(30),str(30),'-srcnodata','-9999','-dstnodata','-9999','-r','sum','-t_srs','EPSG:26910']
subprocess.call(warp_cmd)

0

In [60]:
warp_cmd = ["gdalwarp","-overwrite", TAO_ras, avg_height_ras, "-tr", str(30),str(30),'-srcnodata','-9999','-dstnodata','-9999','-r','average','-t_srs','EPSG:26910']
subprocess.call(warp_cmd)

0

In [61]:
warp_cmd = ["gdalwarp","-overwrite", TAO_ras, max_height_ras, "-tr", str(30),str(30),'-srcnodata','-9999','-dstnodata','-9999','-r','max','-t_srs','EPSG:26910']
subprocess.call(warp_cmd)

0

In [62]:
fVeg_cmd = ['python',gdal_calc, '-A', counts_ras, '--NoDataValue','-9999',
 '--outfile', TAO_density, '--calc="A/(30*30)"','--overwrite']
subprocess.run(fVeg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_TAO_counts_30m.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_TAO_density_30m.tif', '--calc="A/(30*30)"', '--overwrite'], returncode=0)

#### Openness

Normalize open index by number of trees

In [69]:
openness_ind = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_DNC_avgTAOheight_30m.tif'
DNC_30m = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_DNC_30m.tif'
avg_TAO_height = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_avg_TOA_height_30m.tif'

In [80]:
veg_cmd = ['python', gdal_calc, '-A', DNC_30m,'-B', avg_TAO_height, '--NoDataValue','-9999',
 '--outfile', openness_ind, '--calc="A*2/B"','--overwrite']
subprocess.run(veg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_DNC_30m.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/TAO/NCALM_SCB_20201120_avg_TOA_height_30m.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_DNC_avgTAOheight_30m.tif', '--calc="A*2/B"', '--overwrite'], returncode=0)

#### LAI'

In [127]:
LAI = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_LAI_prime.tif'
LAI_30 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_SCB_20201120_LAI_prime_30m.tif'

In [72]:
# dertmine which value to use to scale for max veg height
ncalm_CHM = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM.tif'
CHM_ras = gdal.Open(ncalm_CHM) # open the file 
CHM_arr = CHM_ras.GetRasterBand(1).ReadAsArray() #read the first raster band (in this case we know we are only working with single bands) and read to a 2D array
CHM_ras = None # close rasters

In [73]:
print(np.percentile(CHM_arr,99))

30.981


In [74]:
veg_cmd = ['python',gdal_calc, '-A', ncalm_canopy_density,'-B',ncalm_CHM, '--NoDataValue','-9999',
 '--outfile', LAI, '--calc="A*(B/30.981)"','--overwrite']
subprocess.run(veg_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/fVeg/NCALM_SCB_20201120_canopy_density.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/CHM/NCALM_SCB_20201120_CHM.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_structure/NCALM_20201120_LAI_prime.tif', '--calc="A*(B/30.981)"', '--overwrite'], returncode=0)

In [129]:
warp_cmd = ["gdalwarp","-overwrite", LAI, LAI_30, "-tr", str(30),str(30),'-srcnodata','-9999','-dstnodata','-9999','-r','average','-t_srs','EPSG:6339']
subprocess.call(warp_cmd)

0

#### Vegetation Buffer
Create universal buffer

In [77]:
ncalm_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall.tif'
ncalm_buffer_tall_2020 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall_buffered.tif'

buffer_open_2020 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_open_buffered.tif'
buffer_open_2020_30_sum = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_open_buffered_sum_30m.tif'

In [76]:
proximity_cmd = ['python',gdal_proximity, ncalm_tall, ncalm_buffer_tall_2020,'-values','1','-maxdist','1','-nodata','1','-fixed-buf-val','0']
subprocess.run(proximity_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_proximity.py', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall.tif', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall_buffered.tif', '-values', '1', '-maxdist', '1', '-nodata', '1', '-fixed-buf-val', '0'], returncode=0)

In [78]:
buffer_cmd = ['python',gdal_calc, '-A', ncalm_openORshort,'-B',ncalm_buffer_tall_2020,'--NoDataValue','0',
 '--outfile', buffer_open_2020, '--calc="A*B"','--overwrite']
subprocess.run(buffer_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_openORshort.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall_buffered.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_open_buffered.tif', '--calc="A*B"', '--overwrite'], returncode=0)

In [79]:
warp_cmd = ["gdalwarp","-overwrite", buffer_open_2020, buffer_open_2020_30_sum, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr','30','30','-dstnodata','-9999','-r','sum']
subprocess.call(warp_cmd)

0

## Calculate Snow Depth

Time to bring the snow in... <br>
For each flight, we have two snow depths, filtered at 3 meters and 5 meters to use for open vs. under canopy <br>
We use the general open/short/tall/understory classifications above but refine for each flight

### Define flight/date
We're not quite at the level where we want to create a large all-encompassing function, but adjusting consistent file-naming practices so that we can run processes without having to change whole paths is really helpful.

In [165]:
flight = 'NCALM'
date = '20220321'

In [167]:
tic = time.perf_counter()

### Filter Data
No we filter the snow depths in two critical ways.<br> From Kostadinov et al, 2019 "by excluding areas
and improving the accuracy of the lidar-derived elevation datasets. Slopes greater than 30o and lake/pond water bodies as delineated by the National Hydrography Dataset (high resolution) (https://nhd.usgs.gov/) were excluded from the analysis. We used this slope threshold because the uncertainty in elevation increases dramatically above 30 degrees slope (Takahashi et al., 2005; Tinkham et al., 2012)."

#### Slope and Water Bodies

In [168]:
# Filter out all water bodies and slopes >30 degrees
input_shp = 'SCB/supporting_files/masks/NHDWaterbody_2018_EPSG26910.shp'
slope_30 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_slope_LT30.tif'

counts = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts.tif'
counts_GT10 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT10.tif'

In [204]:
base_name = flight+'_SCB_'+date
snow_depth_LTET3 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/corrected_tif/'+base_name+'_vbc_3.tif'
snow_depth_LTET3_fil = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/corrected_tif/'+base_name+'_vbc_3_fil.tif'
snow_depth_LTET5 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/corrected_tif/'+base_name+'_vbc_5.tif'
snow_depth_LTET5_fil = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/corrected_tif/'+base_name+'_vbc_5_fil.tif'

In [170]:
input_tif = snow_depth_LTET3
input_tif_b = snow_depth_LTET5

In [171]:
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, input_tif]
subprocess.call(burn_command)

0

In [172]:
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, input_tif_b]
subprocess.call(burn_command)

0

In [173]:
# define a counts filter
cound_cmd = ['python',gdal_calc, '-A', counts,'--NoDataValue','0',
 '--outfile', counts_GT10, '--calc="A>10"','--overwrite']
subprocess.run(cound_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT10.tif', '--calc="A>10"', '--overwrite'], returncode=0)

In [174]:
# in this case it's okay that the logical will output a value of 0 for each point <.15 because
# the assumption is that those points are 0 and NAns are not included in the calculations
sd_filter_cmd = ['python',gdal_calc, '-A', snow_depth_LTET3,'-B',slope_30,'-C',counts_GT10,'--NoDataValue','-9999',
 '--outfile', snow_depth_LTET3_fil, '--calc="A*B*C*(A>=0.15)"','--overwrite']
subprocess.run(sd_filter_cmd)

sd_filter_cmd = ['python',gdal_calc, '-A', snow_depth_LTET5,'-B',slope_30,'-C',counts_GT10,'--NoDataValue','-9999',
 '--outfile', snow_depth_LTET5_fil, '--calc="A*B*C*(A>=0.15)"','--overwrite']
subprocess.run(sd_filter_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/corrected_tif/NCALM_SCB_20220321_vbc_5.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/DEM/SCB/NCALM_2014_slope_LT30.tif', '-C', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/counts/NCALM_SCB_20201120_counts_GT10.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/corrected_tif/NCALM_SCB_20220321_vbc_5_fil.tif', '--calc="A*B*C*(A>=0.15)"', '--overwrite'], returncode=0)

### Refine Vegetation Classes

In [205]:
base_name = flight+'_SCB_'+date

snow_depth_LTET3_fil = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/corrected_tif/'+base_name+'_vbc_3_fil.tif'
snow_depth_LTET5_fil = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/corrected_tif/'+base_name+'_vbc_5_fil.tif'
flight_spec_short = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_short.tif'
flight_spec_open = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_open.tif'
flight_spec_understory = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_understory.tif'
flight_spec_tall = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_tall.tif'

Refine vegetation classifications based on snow-on flights. We set a threshold of 30 cm so that if the snow-on flight is <30 cm greater than the snow-off flight, we do not keep that short/understory pixel. 

In [206]:
# nodata has to be 0 because of the multiplying step below
SD_open_cmd = ['python',gdal_calc, '-A', snow_depth_LTET5_fil, '-B', ncalm_short,'-C', CHM_LT1pt5,'--NoDataValue','0',
 '--outfile', flight_spec_short, '--calc="((A-C)>0.3)*B"','--overwrite']
subprocess.run(SD_open_cmd)

SD_open_cmd = ['python',gdal_calc, '-A', flight_spec_short, '-B', ncalm_open,'--hideNoData','--NoDataValue','0',
    '--outfile', flight_spec_open, '--calc="1*logical_or(A>0, B>0)"','--overwrite']
subprocess.run(SD_open_cmd)

SD_tall_cmd = ['python',gdal_calc, '-A', snow_depth_LTET3_fil, '-B', ncalm_understory,'-C', CHM_LT1pt5,'--NoDataValue','0',
 '--outfile', flight_spec_understory, '--calc="((A-C)>0.3)*B"','--overwrite']
subprocess.run(SD_tall_cmd)

SD_tall_cmd = ['python',gdal_calc, '-A', flight_spec_understory, '-B', ncalm_tall,'--hideNoData','--NoDataValue','0',
 '--outfile', flight_spec_tall, '--calc="1*logical_or(A>0, B>0)"','--overwrite']
subprocess.run(SD_tall_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_understory.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20201120/canopy_metrics/veg_classes/NCALM_SCB_20201120_tall.tif', '--hideNoData', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_tall.tif', '--calc="1*logical_or(A>0, B>0)"', '--overwrite'], returncode=0)

### Calculate Snow Depth

In [132]:
# # do not use this cell
# snow_depth_all_pts = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD.tif'
# snow_depth_refined = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD.tif'
# snow_depth_refined_binary = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD_binary.tif'
# snow_depth_refined_binary_30 = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD_binary_30m.tif'

In [207]:

base_name = flight+"_SCB_"+date
snow_depth_all_pts = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD.tif'
snow_depth = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD.tif'
snow_depth_open = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_open.tif'
snow_depth_tall = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_tall.tif'

snow_depth_binary = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_binary.tif'
snow_depth_binary_30 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_binarySum_30m.tif'
snow_depth_binary_open_30 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_open_binarySum_30m.tif'
snow_depth_binary_tall_30 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_tall_binarySum_30m.tif'
snow_depth_binary_open_tall_30 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_open_tall_binarySum_30m.tif'

snow_depth_binaryDen_30 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_binaryDen_30m.tif'


In [208]:
SD_open_cmd = ['python', gdal_calc, '-A', snow_depth_LTET5_fil, '-B', flight_spec_open,'--NoDataValue','-9999',
 '--outfile', snow_depth_open, '--calc="A*B*(A>0.15)"','--overwrite']
subprocess.run(SD_open_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/corrected_tif/NCALM_SCB_20220321_ground_vbc_5_fil.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_open.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_open.tif', '--calc="A*B*(A>0.15)"', '--overwrite'], returncode=0)

In [209]:
SD_tall_cmd = ['python', gdal_calc, '-A', snow_depth_LTET3_fil, '-B', flight_spec_tall,'--NoDataValue','-9999',
 '--outfile', snow_depth_tall, '--calc="A*B*(A>0.15)"','--overwrite']
subprocess.run(SD_tall_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/corrected_tif/NCALM_SCB_20220321_ground_vbc_3_fil.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_tall.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_tall.tif', '--calc="A*B*(A>0.15)"', '--overwrite'], returncode=0)

In [210]:
merge_command = ["python", gdal_merge, "-o", snow_depth_all_pts, snow_depth_open, snow_depth_tall,'-n','-9999','-a_nodata','-9999']
subprocess.call(merge_command)

0

In [211]:
# Create a binary file (snow=1, no snow=0)
sd_filter_cmd = ['python',gdal_calc, '-A', snow_depth_all_pts,'--NoDataValue','0',
 '--outfile', snow_depth_binary, '--calc="A>0"','--overwrite']
subprocess.run(sd_filter_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_binary.tif', '--calc="A>0"', '--overwrite'], returncode=0)

In [212]:
# resample
warp_cmd = ["gdalwarp","-overwrite", snow_depth_binary, snow_depth_binary_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr','30','30','-srcnodata','0','-dstnodata','-9999','-r','sum']
subprocess.call(warp_cmd)

0

In [213]:
# resample
warp_cmd = ["gdalwarp","-overwrite", flight_spec_tall, snow_depth_binary_tall_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr','30','30','-srcnodata','0','-dstnodata','-9999','-r','sum']
subprocess.call(warp_cmd)

0

In [214]:
warp_cmd = ["gdalwarp","-overwrite", flight_spec_open, snow_depth_binary_open_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr','30','30','-srcnodata','0','-dstnodata','-9999','-r','sum']
subprocess.call(warp_cmd)

0

In [215]:
snow_cmd = ['python', gdal_calc, '-A', snow_depth_binary_open_30, '-B', snow_depth_binary_tall_30,'--NoDataValue','-9999',
 '--outfile', snow_depth_binary_open_tall_30, '--calc="A-B"','--overwrite']
subprocess.run(snow_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_open_binarySum_30m.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_tall_binarySum_30m.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_open_tall_binarySum_30m.tif', '--calc="A-B"', '--overwrite'], returncode=0)

In [216]:
binary_cmd = ['python',gdal_calc, '-A', snow_depth_binary_30,'--NoDataValue','-9999',
 '--outfile', snow_depth_binaryDen_30, '--calc="A/900"','--overwrite']
subprocess.run(binary_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_binarySum_30m.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_binaryDen_30m.tif', '--calc="A/900"', '--overwrite'], returncode=0)

### Create Vegetation Buffer
for statistical analysis later <br>
We start with a tall buffer. Starting with the fligth specific tall/open classifications. Search for the values of 1 in the tall raster and classify each value according to it's proximity to the neaerest 1 - however, in this case, the max distance is 1 (creating a 1m buffer around each tree) and values within 1m are assigedn a value of 0. All other pixels are given a value of 1. <br>
This let's us create an "open buffered" raster by multiplying our open raster with the new tall buffer. i.e. anywhere the pixel is open (==1) and the tall buffer indicates an open area (==1)

In [217]:
src_tall = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_tall.tif'
buffer_tall = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_tall_buffered.tif'

src_open = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_open.tif'
buffer_open = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_open_buffered.tif'
buffer_open_30 = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_open_buffered_sum_30m.tif'

In [218]:
proximity_cmd = ['python',gdal_proximity, src_tall, buffer_tall,'-values','1','-maxdist','1','-nodata','1','-fixed-buf-val','0']
subprocess.run(proximity_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_proximity.py', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_tall.tif', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_tall_buffered.tif', '-values', '1', '-maxdist', '1', '-nodata', '1', '-fixed-buf-val', '0'], returncode=0)

In [219]:
buffer_cmd = ['python',gdal_calc, '-A', src_open,'-B',buffer_tall,'--NoDataValue','0',
 '--outfile', buffer_open, '--calc="A*B"','--overwrite']
subprocess.run(buffer_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_open.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_tall_buffered.tif', '--NoDataValue', '0', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_open_buffered.tif', '--calc="A*B"', '--overwrite'], returncode=0)

In [220]:
warp_cmd = ["gdalwarp","-overwrite", buffer_open, buffer_open_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr','30','30','-dstnodata','-9999','-r','sum']
subprocess.call(warp_cmd)

0

## Calculate SWE

Start by making sure our density rasters align with snow depth rasters <br>
from the Lidar Processing workflow we can extract our bounds: '730235.96, 738826.45,4364741.66,4372273.16'

### Align Rasters

In [221]:

base_name = flight+'_SCB_'+date
input_density = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_density/'+'snowPALM_SCB_'+date+'_snowDensity.tif'
output_density = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_density/'+'snowPALM_SCB_'+date+'_snowDensity_warp.tif'

# for now we'll use 03/26/2016 data
# input_snow_density = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_density/snowPALM_SCB_20160326_snowDensity_warp.tif'

In [192]:
# # nodata is 0 to account for strange model outputs (i.e. we don't want SWE=0 where density =0)
# warp_cmd = ["gdalwarp","-overwrite", input_density, output_density, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr','1','1','-dstnodata','0']
# subprocess.call(warp_cmd)

### Calculate SWE

In [222]:
base_name = flight+'_SCB_'+date
res = '30'
input_snow_depth = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD.tif'
input_snow_depth_open = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_open.tif'
input_snow_depth_tall = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_depth/'+base_name+'_SD_tall.tif'

input_snow_density = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/snow_density/'+'snowPALM_SCB_'+date+'_snowDensity_warp.tif'

output_SWE =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE.tif'
output_SWE_30 =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE_30m.tif'
output_SWE_open =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE_open.tif'
output_SWE_open_30 =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE_open_30m.tif'
output_SWE_tall =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE_tall.tif'
output_SWE_tall_30 =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE_tall_30m.tif'

In [223]:
swe_cmd = ['python',gdal_calc, '-A', input_snow_depth,'-B',input_snow_density,'--NoDataValue','-9999',
 '--outfile', output_SWE, '--calc="A*B*100"','--overwrite']
subprocess.run(swe_cmd)

swe_open_cmd = ['python',gdal_calc, '-A', input_snow_depth_open,'-B',input_snow_density,'--NoDataValue','-9999',
 '--outfile', output_SWE_open, '--calc="A*B*100"','--overwrite']
subprocess.run(swe_open_cmd)

swe_tall_cmd = ['python',gdal_calc, '-A', input_snow_depth_tall,'-B',input_snow_density,'--NoDataValue','-9999',
 '--outfile', output_SWE_tall, '--calc="A*B*100"','--overwrite']
subprocess.run(swe_tall_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_depth/NCALM_SCB_20220321_ground_SD_tall.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/snow_density/snowPALM_SCB_20220321_snowDensity_warp.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/SWE/NCALM_SCB_20220321_ground_SWE_tall.tif', '--calc="A*B*100"', '--overwrite'], returncode=0)

In [224]:
# resample
warp_cmd = ["gdalwarp","-overwrite", output_SWE, output_SWE_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr',res,res,'-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)

0

In [225]:
# resample
warp_cmd = ["gdalwarp","-overwrite", output_SWE_open, output_SWE_open_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr',res,res,'-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)
warp_cmd = ["gdalwarp","-overwrite", output_SWE_tall, output_SWE_tall_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr',res,res,'-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)

0

#### Apply Buffer Zones

In [226]:

base_name = flight + '_SCB_' + date
res = '30'
input_SWE = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE.tif'
buffer_open = 'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/veg_classes/'+base_name+'_open_buffered.tif'
SWE_open_buffered =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE_open_buffered.tif'
SWE_open_buffered_30 =  'SCB/Sagehen_lidar/'+flight+'/'+base_name+'/SWE/'+base_name+'_SWE_open_buffered_30m.tif'

In [227]:
swe_open_cmd = ['python',gdal_calc, '-A', input_SWE,'-B',buffer_open,'--NoDataValue','-9999',
 '--outfile', SWE_open_buffered, '--calc="A*B"','--overwrite']
subprocess.run(swe_open_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/SWE/NCALM_SCB_20220321_ground_SWE.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/veg_classes/NCALM_SCB_20220321_ground_open_buffered.tif', '--NoDataValue', '-9999', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20220321/SWE/NCALM_SCB_20220321_ground_SWE_open_buffered.tif', '--calc="A*B"', '--overwrite'], returncode=0)

In [228]:
warp_cmd = ["gdalwarp","-overwrite", SWE_open_buffered, SWE_open_buffered_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr',res,res,'-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)

0

In [229]:
warp_cmd = ["gdalwarp","-overwrite", SWE_open_buffered, SWE_open_buffered_30, "-te", '730235.96', '4364741.66', '738826.9599999999627471', '4372273.16', '-tr',res,res,'-dstnodata','-9999','-r','average']
subprocess.call(warp_cmd)

0

In [201]:
toc = time.perf_counter()

In [202]:
(toc-tic)/60

8.070846448333274